In [ ]:
"import tensorflow as tf\n",
    "from tensorflow.python.training import moving_averages\n",
    "from tensorflow.contrib.framework import add_model_variable\n",
    "from scipy import stats\n",
    "sess = tf.InteractiveSession()\n",
    "training_file = '/home/jeremy/CarND/CarND/CarND-Traffic-Sign-Classifier-Project/train.p'\n",
    "testing_file= '/home/jeremy/CarND/CarND/CarND-Traffic-Sign-Classifier-Project/test.p'\n",
    "validate_file ='/home/jeremy/CarND/CarND/CarND-Traffic-Sign-Classifier-Project/valid.p'\n",
    "\n",
    "\n",
    "with open(training_file, mode='rb') as f:\n",
    "    train = pickle.load(f)\n",
    "with open(testing_file, mode='rb') as f:\n",
    "    test = pickle.load(f)\n",
    "with open(validate_file, mode='rb') as f:\n",
    "    valid = pickle.load(f)\n",
    "\n",
    "X_train, Y_train = train['features'], train['labels']\n",
    "X_test, Y_test = test['features'], test['labels']\n",
    "X_valid, Y_valid =  valid['features'], valid['labels']\n",
    "\n",
    "\n",
    "X_train  = X_train.astype(np.float32)\n",
    "Y_train  = Y_train.astype(np.int32)\n",
    "X_test   = X_test.astype(np.float32)\n",
    "Y_test   = Y_test.astype(np.int32),\n",
    "X_valid  = X_valid.astype(np.float32)\n",
    "Y_valid  = Y_valid.astype(np.int32)\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "\n",
    "## Step 1: Dataset Summary & Exploration\n",
    "\n",
    "The pickled data is a dictionary with 4 key/value pairs:\n",
    "\n",
    "- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).\n",
    "- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.\n",
    "- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.\n",
    "- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**\n",
    "\n",
    "Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. "
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Number of training examples = 34799\n",
      "Number of testing examples = 12630\n",
      "Image data shape = (32, 32, 3)\n",
      "Number of classes = 43\n"
     ]
    }
   ],
   "source": [
    "### Replace each question mark with the appropriate value. \n",
    "### Use python, pandas or numpy methods rather than hard coding the results\n",
    "\n",
    "# TODO: Number of training examples\n",
    "n_train = len(X_train)\n",
    "\n",
    "# TODO: Number of validation examples\n",
    "n_validation = len(X_valid)\n",
    "\n",
    "# TODO: Number of testing examples.\n",
    "n_test = len(X_test)\n",
    "\n",
    "# TODO: What's the shape of an traffic sign image?\n",
    "_, height, width, channel = X_train.shape\n",
    "image_shape = (height, width, channel)\n",
    "# TODO: How many unique classes/labels there are in the dataset.\n",
    "n_classes = 43\n",
    "\n",
    "print(\"Number of training examples =\", n_train)\n",
    "print(\"Number of testing examples =\", n_test)\n",
    "print(\"Image data shape =\", image_shape)\n",
    "print(\"Number of classes =\", n_classes)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Include an exploratory visualization of the dataset"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. \n",
    "\n",
    "The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.\n",
    "\n",
    "**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "12\n"
     ]
    },
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAFwAAABZCAYAAACzIkPrAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz\nAAALEgAACxIB0t1+/AAAD2BJREFUeJztnE2sJNdVx3/n3vro7tfvzbc947GdcWKPERgIHwIkNgiE\nhNgENogsEEiRzIaPSCyIEBKwywLYIhkRiQUSQgKJLCJFEYIFLEicEAFJSLACgYSJPR7Px/vq7qq6\nh8U5t7rfs8fz3rxxzSjTf2mm+1Xdurf61Lnn+5SoKmsMh/Cwb+Bxw5rgA2NN8IGxJvjAWBN8YKwJ\nPjDWBB8YJyK4iPyMiHxVRF4TkY89qJv6Tobcr+MjIhH4GvDTwDeBzwEfVtUvP7jb+85DcYJrfwR4\nTVW/DiAifwl8CLgrwTc3Jnru7KkDx/LjlpUjMdjGK6sKgKZTumZx6Aq/LkRE7OqqiACMJxtIUfkI\nm6vtEgB7e7s0s12bSbuDN6jL+fOcApRldWDlLtl1RSzoVLn+5k22t3eEI+AkBL8M/O/K398EfvTw\nIBF5GXgZ4NyZLX7vox8hoaRg95d3WPQfL6qcmtgPfPLZ5wC4dmfBzhvfsAm1yTPbD5icpoglAM+e\nmwLw0g/9GOXZKwB0YufeurMPwOf/5bO88R+fBaBtdg7cA6kj9AS3BzUqIpcuPgPAvp/b3bsNwOmN\ns2y3yu/+wR/dm1qOkxD8SFDVV4BXAK48c0kTLUkC+I+UQzyu9KdIfqzTQJuCz5d8YjvXbN9iHH2u\nM87VGvo58tZJPr7tFGd25JA0VQTU1nEGJ8RALGoA9mdzAGb+zKNAWS532FFwEoJ/C3hm5e+n/di7\nQEna+VbOHO5n/J5FAsmJGpJ9SqfgHKcpU9DHB6hcBNWjiZ0KBW1rOybF/CCXoiJzr4qJILq2n7s/\nhu84gbxkPdoAoPJ7l1BwXBV4Eivlc8ALIvKciFTALwKfPMF8jwXum8NVtRWRXwM+DUTgE6r6paNc\nK0jPcXnv9hJAtN/OveIKEXHO08xd0bd+EIrCvhel/Zwu0XNtyuLJFV3S1HOl+JfQryO9nMmKu5BI\nWZoeaPZMlnT7JsO3xmeJcaPffUfBiWS4qn4K+NRJ5njc8J4rzcMQBFVZ+fuQEEwJknFz55zXSUD8\nmtBLQfs7olQjU2qxMhneJGhnewAU1QgA7bKSTgSXzyn4sazAdbnzUpbzQRBfq1TfNTH140lhqYCO\ngLVrPzCG5XDNVklC/VmHLP/cMhFSz/PJOTyJom6WhJDNNht1ZnPE01euArBx4UUA7uy3sHsDgPF4\n6tfZLqirQKjtZ0vjsltdP6RASq3P77cVhHlnslt8R0w2zgDQtAUNLcfx1gcluAqoiCnDbA76ubDU\nlP0Pi1mBoabQAD/FqQ0j4Aeef4npBSP4nhvYi8UevXHXmYKr3KR79skr7N28CUDz5tcBKGsTRXt7\nDe3itt+PXR9DQMTE0twfxmL3LQCe2AqM6pJwDDmxFikDY3Clqfl/yd9cOWWRIYHoYia6TSepo/Dx\np6bG2c+/8P0ATC+8wPZ8BkDjn5B6hauNe0hY/GSz3uDFq98LwP/VNtfu/JaNXdyEJnu0plhDoPdW\n1b3dIrpIikoI3dtd1nfBmsMHxuAcjuBmYXa18/G4MsD4oPNzRQlnz5rye+59zwOwed4+b88XNHPj\n3tgrhBIJNl9yU65pjGNVd9kajW38898NwP9822Jw23duIQs3N1NW5pCSxVDqrGZKu56k5lAdw70f\nmOACHo/rvbveHLCtryK9dRI93Hru/BmeOGWKcTI9B8CdmYVrF/vbBHELpzCPsKw3ieWmnXcl2Cws\nMpiaBtzrnI5tzMVzFwHYu3mTW1kstRZdjEEYlaY0t++Ysl3Mbc5psUGMG8cKXq1FysAYlMONv80s\nzEoyh/2y2YcuTcBpZVt366nnoDSRcmdnG4DZzDg2akdwJRZ9TCpPo253FyP3Wn3bd/NtmibH1G2u\n05WZjM+9/7t4zbl///rXAAixoMvhQpcdsXDvNVXmFR9DpKw5fGAMH0sR8YjfQbmXEwtBhKo2rtzY\nOm8nizG3d+4AMPdsS9Cl5xkrk8WpOmufMu5jL4XYLqnGNteCQJqZGbjwtF2OKJ6qJ1y9+hIA/+Wm\nnuxdY7GwcXFsDlLoTFcsJJDadhn5PALWHD4wBrdSRMJqNLxPfWW5vTmuuPz0Fbu5qXHs7d0d5nvb\nfoFnYqLtglhvobVZLh3GzZqW5mbjCxXisZTROeauN7qZWzBZpqdttkamB64+/30AXL8xpW1tsr3r\nb9gwd+1HW5vUkylyDN9+WIKLQKyARMzK0uMT48pu+tLFpzhz0UzAnbkRd7G/Q8zBLbevCxcjWp+j\n81iH9i7hag2AIStNpKIe2QOa+9luZg+z6zoWc1PGG0740dNXef1NI3B9yz5lbOvVVUWMo5WQ8b2x\nFikDY1AODyFST08RaXqVqa15cU+eNo66+PSLzJ1jZ3umKCV1/bYtKhsnzqWtTPrsvvSZ/xWF7KHT\nXIGSAIJzqM8xc63ZzndQTz4zs7Xr8Sbnz1g4tluYg5QWtrtGoaDY2CLE7CUfgQZHHrnGA8GgHB6L\ngtNnzpEWO5Tutm/Uxm0XL10GQMYX2M8K0iN2UYToMWsZmSJdYM5KStLzcx8lOJDyOhiNVAUPq6BZ\nkU7MZGyBxjl7nmMvbDPZOG33ePkKADeuvw5AqXNG03GfFDkKhvU0RajLmjbNGY+saOeJs08BEMe2\nvfeafbIACK4gg0SKYguAmR5UkCEsi4q0Ly5KvK0SqH8GsszQyOE86ZJw/UNT6auxysLIVXsJXiFC\nKKqVCNy9sRYpA2PgnKaiqUHoiB7/mDcWnas8BFqVNV2XQ6leEYXStiZmch1h24uKJTInItKbgW/n\nPe2TGQVmf8/3zfNs59u2O4DSa10mk03E4zKzbRs3chM2hDEhHq/Ubc3hA2Ngx0dRaQlRSME49fa+\ncXZz3coST529wKg2028+t5i0di3Jd0LEnI9i5MWZTCCnjA9UcR3iOuf4KEoZskPlyeSZOzS0FF6L\nOJ3YPRT1hNs7VuMy3zUOH0/Nox2PT1PWk2MpzXuOFJFnROTvReTLIvIlEflNP35WRD4jIv/pn2eO\nvOpjjKNweAv8lqp+QUQ2gc+LyGeAXwH+TlU/7u0mHwN++90mklBQb54mzfeQLIs9TvHmTTPHmvkO\nFy5b+ix51VSz2AcPAaTGuC3ksolSUMxkVFYdEF3530otACpJNB4tbJzDxWV5ESPjsUcXvaxi1s1B\nbEeUE1tn5FHD0WhCWY+XtTVHwD0JrqrXgGv+fVtEvoIV438I+Akf9ufAP3APghdFyflzl1jM92gW\nJiLSnqe0PD22s5hT3LgGwPlLH7BzMqZx8ZILddLcCB81gJeFt054iCspvCWhAdr9t5jv3/AfZ2HX\nwgtDN8Zjxl7ks+8BLU0LNjc8B1pY90ZV2b2GoiSE4r0zC0XkCvADwD8DT/rDAPg28ORdrnlZRF4V\nkVdvegHO44wjK00RmQJ/DXxUVe+smkKqqiLvXJyx2gHx0kvfo2VVIyTq2tiyHpnYSKeMA1MnqCdw\nG4/ibWycZc+37WJ/z+c1DmwXu5RuyhWlc7FMew4vXBy0s6wgbxA621XBFeTGxDh4ND1F0ztMdt1o\nNKH0LH3wvqHCveTcX/TAzUIRKTFi/4Wq/o0ffl1ELvn5S8AbR171McY9OVzs8f0Z8BVV/eOVU58E\nfhn4uH/+7b3mCiEwGY3oytiXKedIX2o9Yte1LGauLL2IsmwWTGpPLuTeHpfhdAu6bDKqcXFVC3ic\npFts+3hTlKQG6TnbFONow2Rzq4GuM7meO+KqatwnqXNUMDcHiMR38qzeFUcRKT8O/BLwbyLyRT/2\nOxih/0pEPgJ8A/iF4y39eOIoVso/cvfn+FPHWSyIMKpLoFzGrHPhu3Nu13bMSpPhsx2v+UsdnXP7\nxE2zXb+8nSXU20taNxmjKOrpttaLdsQ5NwQYe+XVeNOigJ2bk20768MDhZdCxBiRIgfRwoF7toT4\n8fAQSt2C1aeE2P994IZiILniamZ2e6NRTfDakVyVJdGSADsordeoqMdgmtk+qi5yvBMu5wjGo4rp\nKc/uB1OCnSviGJUi2rHSRVgoyxVPcllSDWbjH0dhwjqWMjgGjqUsO3yFvC1zd1oO+EufKKZXThCL\n3J9p42pXfEy32HWR0DinBxqSi5ngTs14ZM7K5umzFB6raVo3J11k1HXVp/BiVpAhLDk6V2/lhLYm\noiwLU4+CNYcPjOHrUkL09u7cObbSUuzQw9+EvvMn9b1BdqquAmHTuHLXD8737zjnwdj7eTY3zfQr\nx1t9G3kOL+RS5hAUxHsxPYRgEU6b6y1X7J/eNkWs7R4/uTntW1GOgodSH44uU2R9SZCuDDikiIRl\nu3YWEUv7QKlKPzad9uckGTE3p5aaq725KpYVc++Zb93y6TyAlopISjm54GtL6IuVbnnz1T8t3JKJ\ngR8k0B3DGF+LlIHxEBpjDX2Otj/e92P3ptZKHreP6CVXkDkNJ+gyVFtnj/AUAa89Kc2cLCrzPFVh\nlFvEx2YCRkwkTCYllb8gIXujqsvC6me8bPk3pjZnKTWXpKJec/iji2H7NDUxn+1QViMkeCfYIRGu\nqr0iDT3TL8ub87jF3FtOZjvU/vKB0YZXRNUlMRpHB19nqSO090y1MVkePDoZNPY9oj0vivQKeuKK\n/rJfH8vAJAhhzeGPLgbl8JQ69vd3Kauqd3iCLK0NyOXLuRJwNXaRMzgHO9xEl1ZNfhdV0KK/NuXX\n/6w6J86p2ZgTd/9Talaau7I1tNQ7mZPz7oldQyiPXlcIQxdzIoxCPLAFc/fB0hKUlcDWMi+ZOPhg\n+o0fItHFRt8+nhKpW77RJ8+b/48ugsa15y1nnvBYLKj6vvq3x3pirmF3pSmaCCEeqz58LVIGxn2/\nt/C+FhO5jvVgvznYoveP8xz9Pt+nqheOMnBQggOIyKuq+sODLnofeK/ucy1SBsaa4APjYRD8lYew\n5v3gPbnPwWX44461SBkYgxH8UX7X+LtUCP++iHxLRL7o/372xGsNIVIe9XeNe+XYpdUKYeDnsFqb\nHVX9wwe11lAc3r9rXFUXQH7X+CMBVb2mql/w79tArhB+4BiK4O/0rvH35AedFIcqhAF+XUT+VUQ+\n8SCaDtZKcwWHK4SBPwHeD3wQq5E/+pvZ74KhCH4f7xofFu9UIayqr6tqp/Yylz/FROOJMBTBH+l3\njd+tQjiXYzt+Hvj3k641SDz8JO8aHwh3qxD+sIh8EAu//zfwqyddaO1pDoy10hwYa4IPjDXBB8aa\n4ANjTfCBsSb4wFgTfGCsCT4w/h+EFqfEsGEWPwAAAABJRU5ErkJggg==\n",
      "text/plain": [
       "<matplotlib.figure.Figure at 0x7f027c1c1320>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "### Data exploration visualization code goes here.\n",
    "### Feel free to use as many code cells as needed.\n",
    "import matplotlib.pyplot as plt\n",
    "# Visualizations will be shown in the notebook.\n",
    "%matplotlib inline\n",
    "\n",
    "index = random.randint(0,len(X_train))\n",
    "\n",
    "image = X_train[index].squeeze()\n",
    "\n",
    "plt.figure(figsize=(1,1))\n",
    "\n",
    "plt.imshow(image)\n",
    "\n",
    "print(Y_train[index])\n",
    "\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAYEAAAD8CAYAAACRkhiPAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz\nAAALEgAACxIB0t1+/AAAEyNJREFUeJzt3X+s3fV93/Hna25Ko2ZopNwhxzYzSE4kg1pHXHlIaSvW\nrMNN0kCmihp1gXYZTgSNiJSpgmxSskqW0NYkU7aFySkI0FKIN5JgNWQrYVFppRJ6yVjAEA8TQPjK\nsV1Q6+yHaE3e++N8DIfre31/nOt7ru/n+ZCO7ve8vz/O537utV/38/l+z/mmqpAk9elvjbsBkqTx\nMQQkqWOGgCR1zBCQpI4ZApLUMUNAkjpmCEhSxwwBSeqYISBJHfuJcTdgPueff35t3rx53M2QpLPK\n448//hdVNTHfdqs+BDZv3szU1NS4myFJZ5UkLy5kO6eDJKljhoAkdcwQkKSOGQKS1DFDQJI6ZghI\nUsfmDYEkm5J8O8nTSfYnubnV357koSTPtq/nDe1za5KDSQ4kuXKoflmSJ9u6LyTJmfm2JEkLsZCR\nwAngk1W1FbgcuCnJVuAW4OGq2gI83J7T1u0ELgF2AF9Msq4d63bgBmBLe+xYxu9FkrRI84ZAVR2u\nqu+25R8BzwAbgKuAu9tmdwNXt+WrgPuq6tWqeh44CGxPsh44t6oercGNje8Z2keSNAaLesdwks3A\nu4HvABdU1eG26ofABW15A/Do0G6HWu1v2vLM+qqz+ZZvnFJ74bb3j6ElknRmLfjEcJK3AfcDn6iq\n48Pr2l/2tVyNSrIryVSSqWPHji3XYSVJMywoBJK8hUEAfLmqvtrKR9oUD+3r0VafBjYN7b6x1abb\n8sz6KapqT1VNVtXkxMS8n38kSVqieaeD2hU8dwDPVNXnhlbtA64HbmtfHxiq/0GSzwHvYHAC+LGq\nei3J8SSXM5hOug74d8v2nSySUz6StLBzAu8BPgw8meSJVvsUg//89yb5CPAicA1AVe1Pshd4msGV\nRTdV1WttvxuBu4C3At9sD0nSmMwbAlX1p8Bc1/O/d459dgO7Z6lPAZcupoGSpDPHdwxLUscMAUnq\nmCEgSR1b9beX1NnFq66ks4sjAUnqmCEgSR1zOkiL4nSPtLY4EpCkjhkCktQxp4MkaYaepj0dCUhS\nxwwBSeqYISBJHTMEJKljhoAkdcyrgzrV09UPkubmSECSOjZvCCS5M8nRJE8N1b6S5In2eOHkbSeT\nbE7y/4bW/cehfS5L8mSSg0m+0O5dLEkao4VMB90F/HvgnpOFqvr1k8tJPgv81dD2z1XVtlmOcztw\nA4ObzD8I7GCN3WPYKRZJZ5t5RwJV9Qjwymzr2l/z1wD3nu4YSdYD51bVo1VVDALl6sU3V5K0nEY9\nJ/ALwJGqenaodlGbCvrjJL/QahuAQ0PbHGo1SdIYjXp10LW8eRRwGLiwql5Ochnw9SSXLPagSXYB\nuwAuvPDCEZsoSZrLkkcCSX4C+MfAV07WqurVqnq5LT8OPAe8E5gGNg7tvrHVZlVVe6pqsqomJyYm\nltpESdI8RpkO+ofA96vq9WmeJBNJ1rXli4EtwA+q6jBwPMnl7TzCdcADI7y2JGkZLOQS0XuBPwPe\nleRQko+0VTs59YTwLwLfa5eM/hfgY1V18qTyjcDvAwcZjBDW1JVBknQ2mvecQFVdO0f9N2ep3Q/c\nP8f2U8Cli2yfJOkM8h3DktQxQ0CSOmYISFLHDAFJ6pghIEkd834CK8gPmJO02jgSkKSOGQKS1DFD\nQJI6ZghIUscMAUnqmCEgSR0zBCSpY4aAJHXMEJCkjhkCktQxQ0CSOraQ20vemeRokqeGap9JMp3k\nifZ439C6W5McTHIgyZVD9cuSPNnWfaHda1iSNEYLGQncBeyYpf75qtrWHg8CJNnK4N7Dl7R9vnjy\nxvPA7cANDG4+v2WOY0qSVtC8IVBVjwCvzLddcxVwX1W9WlXPM7ip/PYk64Fzq+rRqirgHuDqpTZa\nkrQ8Rjkn8PEk32vTRee12gbgpaFtDrXahrY8sz6rJLuSTCWZOnbs2AhNlCSdzlJD4HbgYmAbcBj4\n7LK1CKiqPVU1WVWTExMTy3loSdKQJYVAVR2pqteq6sfAl4DtbdU0sGlo042tNt2WZ9YlSWO0pBBo\nc/wnfQg4eeXQPmBnknOSXMTgBPBjVXUYOJ7k8nZV0HXAAyO0W5K0DOa9vWSSe4ErgPOTHAI+DVyR\nZBtQwAvARwGqan+SvcDTwAngpqp6rR3qRgZXGr0V+GZ7SJLGaN4QqKprZynfcZrtdwO7Z6lPAZcu\nqnWSpDPKdwxLUscMAUnqmCEgSR0zBCSpY4aAJHXMEJCkjhkCktQxQ0CSOmYISFLHDAFJ6ti8Hxsh\nabw23/KNU2ov3Pb+MbREa5EjAUnqmCEgSR1zOmgNcxpB0nwcCUhSxwwBSerYQu4sdifwAeBoVV3a\nav8G+FXgr4HngN+qqr9Mshl4BjjQdn+0qj7W9rmMN+4s9iBwc1XVcn4za5XTOpLOlIWMBO4Cdsyo\nPQRcWlU/C/wv4Nahdc9V1bb2+NhQ/XbgBgb3Hd4yyzElSSts3hCoqkeAV2bU/qiqTrSnjwIbT3eM\ndmP6c6vq0fbX/z3A1UtrsiRpuSzH1UH/FPjK0POLkjwB/BXwL6vqT4ANwKGhbQ612llntU3NrLb2\nSDq7jBQCSf4FcAL4cisdBi6sqpfbOYCvJ7lkCcfdBewCuPDCC0dpoiTpNJZ8dVCS32Rwwvg3Tp7g\nrapXq+rltvw4g5PG7wSmefOU0cZWm1VV7amqyaqanJiYWGoTJUnzWFIIJNkB/A7wwar6v0P1iSTr\n2vLFDE4A/6CqDgPHk1yeJMB1wAMjt16SNJKFXCJ6L3AFcH6SQ8CnGVwNdA7w0OD/9NcvBf1F4HeT\n/A3wY+BjVXXypPKNvHGJ6DfbQ5I0RvOGQFVdO0v5jjm2vR+4f451U8Cli2qdJOmM8h3DktQxQ0CS\nOmYISFLH/ChpncI3oEn9cCQgSR0zBCSpY4aAJHXMEJCkjhkCktQxQ0CSOmYISFLHDAFJ6pghIEkd\n8x3DWrN85/Pa5894dI4EJKljhoAkdcwQkKSOzRsCSe5McjTJU0O1tyd5KMmz7et5Q+tuTXIwyYEk\nVw7VL0vyZFv3hXavYUnSGC1kJHAXsGNG7Rbg4araAjzcnpNkK7ATuKTt88WTN54HbgduYHDz+S2z\nHFOStMLmDYGqegR4ZUb5KuDutnw3cPVQ/b6qerWqngcOAtuTrAfOrapHq6qAe4b2kSSNyVLPCVxQ\nVYfb8g+BC9ryBuCloe0OtdqGtjyzLkkao5FPDLe/7GsZ2vK6JLuSTCWZOnbs2HIeWpI0ZKlvFjuS\nZH1VHW5TPUdbfRrYNLTdxlabbssz67Oqqj3AHoDJycllDRitLb5ZSBrNUkcC+4Dr2/L1wAND9Z1J\nzklyEYMTwI+1qaPjSS5vVwVdN7SPJGlM5h0JJLkXuAI4P8kh4NPAbcDeJB8BXgSuAaiq/Un2Ak8D\nJ4Cbquq1dqgbGVxp9Fbgm+0hSRqjeUOgqq6dY9V759h+N7B7lvoUcOmiWidJOqN8x7AkdcwQkKSO\nGQKS1DFDQJI6ZghIUscMAUnqmCEgSR0zBCSpY4aAJHXMEJCkjhkCktQxQ0CSOmYISFLHDAFJ6pgh\nIEkdMwQkqWOGgCR1bMkhkORdSZ4YehxP8okkn0kyPVR/39A+tyY5mORAkiuX51uQJC3VvLeXnEtV\nHQC2ASRZB0wDXwN+C/h8Vf3e8PZJtgI7gUuAdwDfSvLOoXsQS5JW2HJNB70XeK6qXjzNNlcB91XV\nq1X1PHAQ2L5Mry9JWoLlCoGdwL1Dzz+e5HtJ7kxyXqttAF4a2uZQq0mSxmTkEEjyk8AHgf/cSrcD\nFzOYKjoMfHYJx9yVZCrJ1LFjx0ZtoiRpDssxEvgV4LtVdQSgqo5U1WtV9WPgS7wx5TMNbBrab2Or\nnaKq9lTVZFVNTkxMLEMTJUmzWY4QuJahqaAk64fWfQh4qi3vA3YmOSfJRcAW4LFleH1J0hIt+eog\ngCQ/Dfwy8NGh8r9Osg0o4IWT66pqf5K9wNPACeAmrwySpPEaKQSq6v8APzOj9uHTbL8b2D3Ka0qS\nlo/vGJakjo00EpAWY/Mt3zil9sJt7x9DS3Sm+DM++zgSkKSOGQKS1DFDQJI6ZghIUscMAUnqmCEg\nSR0zBCSpY4aAJHXMN4tJGjvfZDY+jgQkqWOGgCR1zBCQpI4ZApLUMUNAkjrm1UFaFbw6ZGnsN41q\npJFAkheSPJnkiSRTrfb2JA8lebZ9PW9o+1uTHExyIMmVozZekjSa5ZgO+gdVta2qJtvzW4CHq2oL\n8HB7TpKtwE7gEmAH8MUk65bh9SVJS3QmzglcBdzdlu8Grh6q31dVr1bV88BBYPsZeH1J0gKNGgIF\nfCvJ40l2tdoFVXW4Lf8QuKAtbwBeGtr3UKudIsmuJFNJpo4dOzZiEyVJcxn1xPDPV9V0kr8LPJTk\n+8Mrq6qS1GIPWlV7gD0Ak5OTi95fkrQwI4VAVU23r0eTfI3B9M6RJOur6nCS9cDRtvk0sGlo942t\nJmkMZl5Z5FVFfVrydFCSn07yt08uA/8IeArYB1zfNrseeKAt7wN2JjknyUXAFuCxpb6+JGl0o4wE\nLgC+luTkcf6gqv5rkj8H9ib5CPAicA1AVe1Pshd4GjgB3FRVr43UeknSSJYcAlX1A+DnZqm/DLx3\njn12A7uX+prS2cw3dmk18mMjJKljhoAkdcwQkKSOGQKS1DFDQJI6ZghIUse8n4CkRfFS17XFkYAk\ndcwQkKSOOR20SjjEnpt90zd//meWIwFJ6pghIEkdczpIWkZOXawu/jzm50hAkjpmCEhSx5wOkmZw\nCkFLdTb+7oxye8lNSb6d5Okk+5Pc3OqfSTKd5In2eN/QPrcmOZjkQJIrl+MbkCQt3SgjgRPAJ6vq\nu+1ew48neait+3xV/d7wxkm2AjuBS4B3AN9K8k5vMSlJ4zPK7SUPA4fb8o+SPANsOM0uVwH3VdWr\nwPNJDgLbgT9bahukcTgbh/xaPmvt578sJ4aTbAbeDXynlT6e5HtJ7kxyXqttAF4a2u0Qpw8NSdIZ\nNnIIJHkbcD/wiao6DtwOXAxsYzBS+OwSjrkryVSSqWPHjo3aREnSHEa6OijJWxgEwJer6qsAVXVk\naP2XgD9sT6eBTUO7b2y1U1TVHmAPwOTkZI3SRulssNqmGFZbe3TmjHJ1UIA7gGeq6nND9fVDm30I\neKot7wN2JjknyUXAFuCxpb6+JGl0o4wE3gN8GHgyyROt9ing2iTbgAJeAD4KUFX7k+wFnmZwZdFN\nXhkkSeM1ytVBfwpkllUPnmaf3cDupb6mpIVbbVM6q609K+103/84+8aPjZCkjhkCktSxNf3ZQb0P\nPzU3fzfk78CAIwFJ6pghIEkdMwQkqWOGgCR1zBCQpI4ZApLUMUNAkjpmCEhSxwwBSeqYISBJHTME\nJKljhoAkdcwQkKSOGQKS1LEVD4EkO5IcSHIwyS0r/fqSpDesaAgkWQf8B+BXgK0M7ke8dSXbIEl6\nw0qPBLYDB6vqB1X118B9wFUr3AZJUrPSIbABeGno+aFWkySNQapq5V4s+TVgR1X9s/b8w8Dfr6rf\nnrHdLmBXe/ou4MASXu584C9GaG4P7KP52Ufzs4/mN44++ntVNTHfRit9j+FpYNPQ842t9iZVtQfY\nM8oLJZmqqslRjrHW2Ufzs4/mZx/NbzX30UpPB/05sCXJRUl+EtgJ7FvhNkiSmhUdCVTViSS/Dfw3\nYB1wZ1XtX8k2SJLesNLTQVTVg8CDK/BSI00ndcI+mp99ND/7aH6rto9W9MSwJGl18WMjJKljay4E\n/FiK2SW5M8nRJE8N1d6e5KEkz7av542zjeOUZFOSbyd5Osn+JDe3un3UJPmpJI8l+Z+tj/5Vq9tH\nMyRZl+R/JPnD9nzV9tGaCgE/luK07gJ2zKjdAjxcVVuAh9vzXp0APllVW4HLgZva74599IZXgV+q\nqp8DtgE7klyOfTSbm4Fnhp6v2j5aUyGAH0sxp6p6BHhlRvkq4O62fDdw9Yo2ahWpqsNV9d22/CMG\n/4A3YB+9rgb+d3v6lvYo7KM3SbIReD/w+0PlVdtHay0E/FiKxbmgqg635R8CF4yzMatFks3Au4Hv\nYB+9SZvmeAI4CjxUVfbRqf4t8DvAj4dqq7aP1loIaIlqcJlY95eKJXkbcD/wiao6PrzOPoKqeq2q\ntjF4t//2JJfOWN91HyX5AHC0qh6fa5vV1kdrLQQW9LEUet2RJOsB2tejY27PWCV5C4MA+HJVfbWV\n7aNZVNVfAt9mcJ7JPnrDe4APJnmBwXT0LyX5T6ziPlprIeDHUizOPuD6tnw98MAY2zJWSQLcATxT\nVZ8bWmUfNUkmkvydtvxW4JeB72Mfva6qbq2qjVW1mcH/P/+9qv4Jq7iP1tybxZK8j8Gc3MmPpdg9\n5iatCknuBa5g8GmGR4BPA18H9gIXAi8C11TVzJPHXUjy88CfAE/yxlzupxicF7CPgCQ/y+Ck5joG\nf0DurarfTfIz2EenSHIF8M+r6gOruY/WXAhIkhZurU0HSZIWwRCQpI4ZApLUMUNAkjpmCEhSxwwB\nSeqYISBJHTMEJKlj/x8Uav8QckEgiwAAAABJRU5ErkJggg==\n",
      "text/plain": [
       "<matplotlib.figure.Figure at 0x7f027c115080>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "# histogram of label frequency\n",
    "hist, bins = np.histogram(Y_train, bins=n_classes)\n",
    "width = 0.7 * (bins[1] - bins[0])\n",
    "center = (bins[:-1] + bins[1:]) / 2\n",
    "plt.bar(center, hist, align='center', width=width)\n",
    "plt.show()\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "----\n",
    "\n",
    "## Step 2: Design and Test a Model Architecture\n",
    "\n",
    "Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).\n",
    "\n",
    "The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! \n",
    "\n",
    "With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. \n",
    "\n",
    "There are various aspects to consider when thinking about this problem:\n",
    "\n",
    "- Neural network architecture (is the network over or underfitting?)\n",
    "- Play around preprocessing techniques (normalization, rgb to grayscale, etc)\n",
    "- Number of examples per label (some have more than others).\n",
    "- Generate fake data.\n",
    "\n",
    "Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Pre-process the Data Set (normalization, grayscale, etc.)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. \n",
    "\n",
    "Other pre-processing steps are optional. You can try different techniques to see if it improves performance. \n",
    "\n",
    "Use the code cell (or multiple code cells, if necessary) to implement the first step of your project."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Training RGB shape: (34799, 32, 32, 3)\n",
      "\n",
      "Mean of Traing RGB values : -1.00277\n",
      "\n",
      "Data needs to be normalized so that it is more closely centered at 0\n",
      "Subtracting the mean of the grayscale value from from the orignial dataset creates a mean that is effectively 0\n",
      " \n"
     ]
    }
   ],
   "source": [
    "### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include \n",
    "### converting to grayscale, etc.\n",
    "### Feel free to use as many code cells as needed.\n",
    "\n",
    "\n",
    "\n",
    "print('Training RGB shape:', X_train.shape)\n",
    "#print('Testing Grayscale shape:', X_train_gray.shape)\n",
    "\n",
    "#print('RGB shape:', X_train_rgb.shape)\n",
    "#print('Grayscale shape:', X_train_gray.shape)\n",
    "print()\n",
    "print('Mean of Traing RGB values :', np.mean(X_train))\n",
    "#print ('Mean of Training Grayscale Values', np.mean(X_train_gray))\n",
    "print()\n",
    "print('Data needs to be normalized so that it is more closely centered at 0')\n",
    "print('Subtracting the mean of the grayscale value from from the orignial dataset creates a mean that is effectively 0')\n",
    "print(' ')\n",
    "#clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))\n",
    "\n",
    "\n",
    "\n",
    "X_train = samplewise_norm(X_train)\n",
    "#X_train = clahe.apply(X_train)\n",

    "#print(' ')\n",
    "#print( np.mean(X_train))\n",
    "\n",
    "\n",
    "\n",
    "                                             \n",
    "\n",
    "X_train, Y_train = shuffle(X_train, Y_train)\n",
    "\n",
    "EPOCHS = 15\n",
    "\n",
    "BATCH_SIZE = 256\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Model Architecture"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "metadata": {
    "collapsed": true
   },
   "outputs": [],
   "source": [
    "### Define your architecture here.\n",
    "### Feel free to use as many code cells as needed.\n",
    "\n",
    "from tensorflow.contrib.layers import flatten\n",
    "\n",
    "def LeNet(x):    \n",
    "    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer\n",
    "    mu = 0\n",
    "    sigma = 0.1\n",
    "    \n",
    "    # SOLUTION: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.\n",
    "    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 6), mean = mu, stddev = sigma))\n",
    "    conv1_b = tf.Variable(tf.zeros(6))\n",
    "    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b\n",
    "\n",
    "    # SOLUTION: Activation.\n",
    "    conv1 = tf.nn.relu(conv1)\n",
    "\n",
    "    # SOLUTION: Pooling. Input = 28x28x6. Output = 14x14x6.\n",
    "    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')\n",
    "\n",
    "    # SOLUTION: Layer 2: Convolutional. Output = 10x10x16.\n",
    "    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))\n",
    "    conv2_b = tf.Variable(tf.zeros(16))\n",
    "    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b\n",
    "    \n",
    "    # SOLUTION: Activation.\n",
    "    conv2 = tf.nn.relu(conv2)\n",
    "\n",
    "    # SOLUTION: Pooling. Input = 10x10x16. Output = 5x5x16.\n",
    "    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')\n",
    "\n",
    "    # SOLUTION: Flatten. Input = 5x5x16. Output = 400.\n",
    "    fc0   = flatten(conv2)\n",
    "    \n",
    "    # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.\n",
    "    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))\n",
    "    fc1_b = tf.Variable(tf.zeros(120))\n",
    "    fc1   = tf.matmul(fc0, fc1_W) + fc1_b\n",
    "    \n",
    "    # SOLUTION: Activation.\n",
    "    fc1    = tf.nn.relu(fc1)\n",
    "\n",
    "    # SOLUTION: Layer 4: Fully Connected. Input = 120. Output = 84.\n",
    "    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))\n",
    "    fc2_b  = tf.Variable(tf.zeros(84))\n",
    "    fc2    = tf.matmul(fc1, fc2_W) + fc2_b\n",
    "    \n",
    "    # SOLUTION: Activation.\n",
    "    fc2    = tf.nn.relu(fc2)\n",
    "\n",
    "    # SOLUTION: Layer 5: Fully Connected. Input = 84. Output = 10.\n",
    "    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, 43), mean = mu, stddev = sigma))\n",
    "    fc3_b  = tf.Variable(tf.zeros(43))\n",
    "    logits = tf.matmul(fc2, fc3_W) + fc3_b\n",
    "    \n",
    "    return logits"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Train, Validate and Test the Model"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation\n",
    "sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Training...\n",
      "\n",
      "EPOCH 1 ...\n",
      "Validation Accuracy = 0.048\n",
      "\n",
      "EPOCH 2 ...\n",
      "Validation Accuracy = 0.049\n",
      "\n",
      "EPOCH 3 ...\n",
      "Validation Accuracy = 0.048\n",
      "\n",
      "EPOCH 4 ...\n",
      "Validation Accuracy = 0.048\n",
      "\n"
     ]
    }
   ],
   "source": [
    "### Train your model here.\n",
    "### Calculate and report the accuracy on the training and validation set.\n",
    "### Once a final model architecture is selected, \n",
    "### the accuracy on the test set should be calculated and reported as well.\n",
    "### Feel free to use as many code cells as needed.\n",
    "\n",
    "x = tf.placeholder(tf.float32, (None, 32, 32, 3))\n",
    "y = tf.placeholder(tf.int32, (None))\n",
    "one_hot_y = tf.one_hot(y, 43)\n",
    "\n",
    "#Train model\n",
    "\n",
    "rate = 0.001\n",
    "\n",
    "logits = LeNet(x)\n",
    "cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)\n",
    "loss_operation = tf.reduce_mean(cross_entropy)\n",
    "optimizer = tf.train.AdamOptimizer(learning_rate = rate)\n",
    "training_operation = optimizer.minimize(loss_operation)\n",
    "\n",
    "#Evaluate Model\n",
    "correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))\n",
    "accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))\n",
    "saver = tf.train.Saver()\n",
    "\n",
    "def evaluate(X_data, y_data):\n",
    "    num_examples = len(X_data)\n",
    "    total_accuracy = 0\n",
    "    sess = tf.get_default_session()\n",
    "    for offset in range(0, num_examples, BATCH_SIZE):\n",
    "        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]\n",
    "        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})\n",
    "        total_accuracy += (accuracy * len(batch_x))\n",
    "    return total_accuracy / num_examples\n",
    "\n",
    "with tf.Session() as sess:\n",
    "    sess.run(tf.global_variables_initializer())\n",
    "    num_examples = len(X_train)\n",
    "\n",
    "    print(\"Training...\")\n",
    "    print()\n",
    "    for i in range(EPOCHS):\n",
    "        X_train, Y_train = shuffle(X_train, Y_train)\n",
    "        for offset in range(0, num_examples, BATCH_SIZE):\n",
    "            end = offset + BATCH_SIZE\n",
    "            batch_x, batch_y = X_train[offset:end], Y_train[offset:end]\n",
    "            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})\n",
    "\n",
    "        validation_accuracy = evaluate(X_valid, Y_valid)\n",
    "        print(\"EPOCH {} ...\".format(i+1))\n",
    "        print(\"Validation Accuracy = {:.3f}\".format(validation_accuracy))\n",
    "        print()\n",
    "\n",
    "    saver.save(sess, './lenet')\n",
    "    print(\"Model saved\")\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "collapsed": true
   },
   "outputs": [],
   "source": [
    "#Testing Model\n",
    "\n",
    "with tf.Session() as sess:\n",
    "    saver.restore(sess, tf.train.latest_checkpoint('.'))\n",
    "\n",
    "    test_accuracy = evaluate(X_test, y_test)\n",
    "    print(\\\"Test Accuracy = {:.3f}\\\".format(test_accuracy))\n",
    "    "
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "\n",
    "## Step 3: Test a Model on New Images\n",
    "\n",
    "To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.\n",
    "\n",
    "You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Load and Output the Images"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "collapsed": true
   },
   "outputs": [],
   "source": [
    "### Load the images and plot them here.\n",
    "### Feel free to use as many code cells as needed."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Predict the Sign Type for Each Image"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "collapsed": true
   },
   "outputs": [],
   "source": [
    "### Run the predictions here and use the model to output the prediction for each image.\n",
    "### Make sure to pre-process the images with the same pre-processing pipeline used earlier.\n",
    "### Feel free to use as many code cells as needed."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Analyze Performance"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "collapsed": true
   },
   "outputs": [],
   "source": [
    "### Calculate the accuracy for these 5 new images. \n",
    "### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Output Top 5 Softmax Probabilities For Each Image Found on the Web"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. \n",
    "\n",
    "The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.\n",
    "\n",
    "`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.\n",
    "\n",
    "Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:\n",
    "\n",
    "```\n",
    "# (5, 6) array\n",
    "a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,\n",
    "         0.12789202],\n",
    "       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,\n",
    "         0.15899337],\n",
    "       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,\n",
    "         0.23892179],\n",
    "       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,\n",
    "         0.16505091],\n",
    "       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,\n",
    "         0.09155967]])\n",
    "```\n",
    "\n",
    "Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:\n",
    "\n",
    "```\n",
    "TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],\n",
    "       [ 0.28086119,  0.27569815,  0.18063401],\n",
    "       [ 0.26076848,  0.23892179,  0.23664738],\n",
    "       [ 0.29198961,  0.26234032,  0.16505091],\n",
    "       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],\n",
    "       [0, 1, 4],\n",
    "       [0, 5, 1],\n",
    "       [1, 3, 5],\n",
    "       [1, 4, 3]], dtype=int32))\n",
    "```\n",
    "\n",
    "Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "collapsed": true
   },
   "outputs": [],
   "source": [
    "### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. \n",
    "### Feel free to use as many code cells as needed."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Project Writeup\n",
    "\n",
    "Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. "
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \\n\",\n",
    "    \"**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "\n",
    "## Step 4 (Optional): Visualize the Neural Network's State with Test Images\n",
    "\n",
    " This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.\n",
    "\n",
    " Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.\n",
    "\n",
    "For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.\n",
    "\n",
    "<figure>\n",
    " <img src=\"visualize_cnn.png\" width=\"380\" alt=\"Combined Image\" />\n",
    " <figcaption>\n",
    " <p></p> \n",
    " <p style=\"text-align: center;\"> Your output should look something like this (above)</p> \n",
    " </figcaption>\n",
    "</figure>\n",
    " <p></p> \n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "collapsed": true
   },
   "outputs": [],
   "source": [
    "### Visualize your network's feature maps here.\n",
    "### Feel free to use as many code cells as needed.\n",
    "\n",
    "# image_input: the test image being fed into the network to produce the feature maps\n",
    "# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer\n",
    "# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output\n",
    "# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry\n",
    "\n",
    "def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):\n",
    "    # Here make sure to preprocess your image_input in a way your network expects\n",
    "    # with size, normalization, ect if needed\n",
    "    # image_input =\n",
    "    # Note: x should be the same name as your network's tensorflow data placeholder variable\n",
    "    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function\n",
    "    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})\n",
    "    featuremaps = activation.shape[3]\n",
    "    plt.figure(plt_num, figsize=(15,15))\n",
    "    for featuremap in range(featuremaps):\n",
    "        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column\n",
    "        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number\n",
    "        if activation_min != -1 & activation_max != -1:\n",
    "            plt.imshow(activation[0,:,:, featuremap], interpolation=\"nearest\", vmin =activation_min, vmax=activation_max, cmap=\"gray\")\n",
    "        elif activation_max != -1:\n",
    "            plt.imshow(activation[0,:,:, featuremap], interpolation=\"nearest\", vmax=activation_max, cmap=\"gray\")\n",
    "        elif activation_min !=-1:\n",
    "            plt.imshow(activation[0,:,:, featuremap], interpolation=\"nearest\", vmin=activation_min, cmap=\"gray\")\n",
    "        else:\n",
    "            plt.imshow(activation[0,:,:, featuremap], interpolation=\"nearest\", cmap=\"gray\")"
   ]
  }
 ],
 "metadata": {
  "anaconda-cloud": {},
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.5.2"
  },
  "widgets": {
   "state": {},
   "version": "1.1.2"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 1
}